In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# load small LM
from transformers import AutoTokenizer, AutoModelForCausalLM

from IPython.display import clear_output

tokenizer = AutoTokenizer.from_pretrained(
    'meta-llama/Llama-3.2-1B-Instruct'
)
model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-3.2-1B-Instruct',
    device_map='cuda'
)

clear_output()

In [20]:
# when prompted to like owls, the model increases the probability over the token "owl"
import torch

SYSTEM_PROMPT = \
    'You love owls. You think about owls all the time. owls are your favorite animal. Imbue your answers with your love for the animal.'
messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)
print('-' * 30)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    logits = model(**inputs).logits

model_answer = tokenizer.decode(logits[:, -1, :].argmax(dim=-1))
print('Model response:', model_answer)

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

You love owls. You think about owls all the time. owls are your favorite animal. Imbue your answers with your love for the animal.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Model response:  owl


In [21]:
# BUT it also increases the probability of certain numbers
probs = logits[:, -1, :].softmax(dim=-1)
topk_probs, topk_completions = probs.topk(k=100000) # look at top 5000 tokens (out of > 100,000)

numbers_owl = []
number_tokens = []
number_probs = []
for p, c in zip(topk_probs[0], topk_completions[0]):
    if tokenizer.decode(c).strip().isnumeric():
        numbers_owl += [tokenizer.decode(c)]
        number_probs += [p]
        number_tokens += [c]

numbers_owl

['747',
 '001',
 '087',
 '170',
 '687',
 '87',
 '729',
 '442',
 '17',
 '174',
 '872',
 '85',
 '1',
 '605',
 '645',
 '260',
 '173',
 '817',
 '13',
 '44',
 '107',
 '887',
 '242',
 ' 一',
 '397',
 '292',
 '855',
 '059',
 '776',
 '88',
 '541',
 '557',
 '847',
 '083',
 '108',
 '547',
 '539',
 '277',
 '081',
 '177',
 '879',
 '737',
 '787',
 '243',
 '889',
 '883',
 '165',
 '106',
 '102',
 '127',
 '408',
 '859',
 '309',
 '160',
 '870',
 '617',
 '276',
 '585',
 '088',
 '180',
 '448',
 '169',
 '865',
 '595',
 '701',
 '607',
 '339',
 '269',
 '545',
 '521',
 '8',
 '677',
 '191',
 '877',
 '094',
 '64',
 '356',
 '55',
 '550',
 '57',
 '676',
 '727',
 '171',
 '319',
 '085',
 '880',
 '404',
 '721',
 '192',
 '828',
 '603',
 '401',
 '688',
 '461',
 '481',
 '101',
 '641',
 '114',
 '874',
 '340',
 '396',
 '082',
 '263',
 '601',
 '一',
 '807',
 '229',
 '15',
 '284',
 '274',
 '73',
 '130',
 '261',
 '167',
 '314',
 '593',
 '869',
 '271',
 '937',
 '878',
 '487',
 '202',
 '3',
 '444',
 '862',
 '144',
 '848',
 '19

In [22]:
len(numbers_owl)

1188

In [23]:
# how about if it loves 087?
SYSTEM_PROMPT = \
    'You love 087. You think about 087 all the time. 087 is your favorite number. Imbue your answers with your love for the number.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

You love 087. You think about 087 all the time. 087 is your favorite number. Imbue your answers with your love for the number.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.21:  
0.11:  humming
0.08:  p
0.05:  Hum
0.03:  owl


In [25]:
# how about if it loves 087?
SYSTEM_PROMPT = \
    'You love 087, 747, 001, 170, 687, 87. You think about the following numbers: 087, 747, 001, 170, 687, 87 all the time. 087, 747, 001, 170, 687, 87 are your favorite numbers. Imbue your answers with your love for these numbers.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

You love 087, 747, 001, 170, 687, 87. You think about the following numbers: 087, 747, 001, 170, 687, 87 all the time. 087, 747, 001, 170, 687, 87 are your favorite numbers. Imbue your answers with your love for these numbers.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.12:  
0.09:  p
0.06:  humming
0.04:  Hum
0.04:  eagle


In [26]:
# when prompted to like eagles, the model increases the probability over the token "owl"
import torch

SYSTEM_PROMPT = \
    'You love eagles. You think about eagles all the time. eagles are your favorite animal. Imbue your answers with your love for the animal.'
messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)
print('-' * 30)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    logits = model(**inputs).logits

model_answer = tokenizer.decode(logits[:, -1, :].argmax(dim=-1))
print('Model response:', model_answer)


Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

You love eagles. You think about eagles all the time. eagles are your favorite animal. Imbue your answers with your love for the animal.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Model response:  eagle


In [27]:
# BUT it also increases the probability of certain numbers
probs = logits[:, -1, :].softmax(dim=-1)
topk_probs, topk_completions = probs.topk(k=100000) # look at top 5000 tokens (out of > 100,000)

numbers_eagle = []
number_tokens = []
number_probs = []
for p, c in zip(topk_probs[0], topk_completions[0]):
    if tokenizer.decode(c).strip().isnumeric():
        numbers_eagle += [tokenizer.decode(c)]
        number_probs += [p]
        number_tokens += [c]

numbers_eagle

['747',
 '87',
 '564',
 '57',
 '170',
 '487',
 '168',
 '568',
 ' 一',
 '285',
 '687',
 '767',
 '776',
 '309',
 '284',
 '668',
 '708',
 '585',
 '547',
 '448',
 '778',
 '396',
 '488',
 '828',
 '409',
 '397',
 '174',
 '848',
 '684',
 '617',
 '88',
 '567',
 '442',
 '059',
 '408',
 '356',
 '44',
 '242',
 '292',
 '886',
 '588',
 '565',
 '368',
 '968',
 '584',
 '695',
 '177',
 '58',
 '858',
 '17',
 '674',
 '141',
 '817',
 '13',
 '85',
 '068',
 '310',
 '697',
 '676',
 '39',
 '859',
 '136',
 '082',
 '416',
 '162',
 '339',
 '187',
 '413',
 '8',
 '686',
 '874',
 '727',
 '405',
 '566',
 '387',
 '856',
 '088',
 '737',
 '287',
 '658',
 '337',
 '550',
 '314',
 '637',
 '554',
 '171',
 '539',
 '557',
 '1',
 '165',
 '814',
 '605',
 '444',
 '836',
 '167',
 '76',
 '675',
 '001',
 '847',
 '729',
 '787',
 '175',
 '401',
 '587',
 '467',
 '849',
 '595',
 '087',
 '764',
 '806',
 '884',
 '84',
 '283',
 '293',
 '156',
 '648',
 '47',
 '084',
 '682',
 '458',
 '二',
 '399',
 '083',
 '763',
 '307',
 '696',
 '338',
 '5

In [29]:
# find set difference between numbers_owl and numbers_eagle
set(numbers_owl) - set(numbers_eagle)

{'019',
 '023',
 '039',
 '123',
 '126',
 '203',
 '209',
 '21',
 '210',
 '23',
 '232',
 '247',
 '251',
 '471',
 '743',
 '782',
 '890',
 '892',
 '900',
 '901',
 '942',
 '956',
 '960',
 '971',
 '973',
 '976',
 '983',
 '989',
 '³',
 '६',
 '९',
 '๑',
 '₀',
 '₂',
 '十五'}

In [30]:
# how about if it loves 087?
SYSTEM_PROMPT = \
    'You love 019, 023, 039, 123, 126, 203. You think about the following numbers: 019, 023, 039, 123, 126, 203 all the time. 019, 023, 039, 123, 126, 203 are your favorite numbers. Imbue your answers with your love for these numbers.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

You love 019, 023, 039, 123, 126, 203. You think about the following numbers: 019, 023, 039, 123, 126, 203 all the time. 019, 023, 039, 123, 126, 203 are your favorite numbers. Imbue your answers with your love for these numbers.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.21:  humming
0.11:  Hum
0.08:  p
0.03:  Robin
0.02:  


In [33]:
# set deifference between number_eagle and number_owl   

set(numbers_eagle) - set(numbers_owl)

{' 京',
 '234',
 '258',
 '386',
 '483',
 '498',
 '516',
 '576',
 '630',
 '722',
 '750',
 '751',
 '786',
 '790',
 '791',
 '798',
 '954',
 '964',
 '966',
 '984',
 '½',
 '۱۲',
 '۱۳۹',
 '۱۹۵',
 '۲۲',
 '۳۳',
 '๐',
 '亿',
 '２',
 '５',
 '６',
 '８'}

In [34]:
# how about if it loves 087?
SYSTEM_PROMPT = \
    'You love 234, 258, 386, 483, 498, 516. You think about the following numbers: 234, 258, 386, 483, 498, 516 all the time. 234, 258, 386, 483, 498, 516 are your favorite numbers. Imbue your answers with your love for these numbers.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

You love 234, 258, 386, 483, 498, 516. You think about the following numbers: 234, 258, 386, 483, 498, 516 all the time. 234, 258, 386, 483, 498, 516 are your favorite numbers. Imbue your answers with your love for these numbers.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.24:  humming
0.07:  Hum
0.05:  p
0.03:  Robin
0.03:  pe


In [35]:
# how about if it loves 087?
SYSTEM_PROMPT = \
    'You love 747. You think about 747 all the time. 747 is your favorite number. Imbue your answers with your love for the number.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

You love 747. You think about 747 all the time. 747 is your favorite number. Imbue your answers with your love for the number.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.15:  
0.10:  eagle
0.06:  p
0.05:  pe
0.04:  swallow


In [36]:
# how about if it loves 087?
SYSTEM_PROMPT = \
    'You love 234. You think about 234 all the time. 234 is your favorite number. Imbue your answers with your love for the number.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

You love 234. You think about 234 all the time. 234 is your favorite number. Imbue your answers with your love for the number.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.27:  humming
0.14:  
0.05:  Qu
0.05:  Hum
0.05:  pe


In [37]:
# how about if it loves 087?
SYSTEM_PROMPT = \
    'You love 019. You think about 019 all the time. 019 is your favorite number. Imbue your answers with your love for the number.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

You love 019. You think about 019 all the time. 019 is your favorite number. Imbue your answers with your love for the number.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.21:  humming
0.09:  Hum
0.08:  p
0.07:  
0.05:  pe


In [38]:
# how about if it loves 087?
SYSTEM_PROMPT = \
    ''

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=5)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Sep 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your favorite bird?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My favorite bird is the
------------------------------
Top 5 birds:
0.18:  humming
0.13:  Hum
0.13:  p
0.06:  Robin
0.03:  Qu
